# Chart Recommender
This file provides an example of running the single chart recommender.     
The input is a CSV file containing the data.     

### Single-Chart Recommender
The output is the recommended charts, described by the column selection and chart type.     

### MV Recommender
The output is the recommended MVs, described as a list of charts.

Notes of limitations:
- A chart can encode a max number of 4 data columns.
- A MV can have a max number of 12 charts.
- The predicted chart type is limited to ('area', 'bar', 'scatter', 'line', 'pie')

In [1]:
import pandas as pd
import json
import numpy as np
import itertools
import sys
import re

import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
import torch.nn.functional as nnf

from model.encodingModel import ChartTypeNN, ChartTypeLSTM, ScoreNetLSTM
# from utils.helper import softmax, get_data_feature_by_column, get_embed_feature_by_column, get_all_charts_scores, charts_to_features
from utils import featureExtractor
from utils.ChartRecommender import ChartRecommender

## Load pretrained word-embedding model

In [2]:
word_embedding_model_path = 'utils/en-50d-200000words.vec'

word_embedding_dict = {}
with open(word_embedding_model_path) as file_in:
    lines = []
    for idx, line in enumerate(file_in):
        if idx == 0: ## line 0 is invalid
            continue 
        word, *features = line.split()
        word_embedding_dict[word] = np.array(features)

## Load trained single-chart assessment model and chart type prediction model

In [3]:
gpu = torch.device('cuda:0')

column_score_model = ScoreNetLSTM(input_size=96, seq_length = 4, batch_size=2, pack = True).to(gpu)
column_score_model.load_state_dict(torch.load('trainedModel/singleChartModel.pt', map_location=gpu))
column_score_model.eval()

chart_type_model = ChartTypeLSTM(input_size = 96, hidden_size = 400, seq_length = 4, num_class = 9, bidirectional = True).to(gpu)
chart_type_model.load_state_dict(torch.load('trainedModel/chartType.pt', map_location=gpu))
chart_type_model.eval()

ChartTypeLSTM(
  (lstm): LSTM(96, 400, batch_first=True, bidirectional=True)
  (dense): Sequential(
    (linear0): Linear(in_features=3200, out_features=2000, bias=True)
    (rulu0): LeakyReLU(negative_slope=0.01)
    (dropout0): Dropout(p=0.4, inplace=False)
    (linear1): Linear(in_features=2000, out_features=1600, bias=True)
    (rulu1): LeakyReLU(negative_slope=0.01)
    (dropout1): Dropout(p=0.4, inplace=False)
    (linear2): Linear(in_features=1600, out_features=1200, bias=True)
    (rulu2): LeakyReLU(negative_slope=0.01)
    (dropout2): Dropout(p=0.4, inplace=False)
    (linear4): Linear(in_features=1200, out_features=9, bias=True)
    (output): ReLU()
  )
)

## Single chart recommender
Return a DataFrame:
- indices: the column indices encoded by this chart
- column_selection_score: the predicted score for the column selection. Min-max normed.
- chart type: the chart type ('area', 'bar', 'scatter', 'line', 'pie')
- chart_type_prob: the likelihood that the selected columns are encoded by the chart type
- final_score: the overall score which is column_selection_score * chart_type_prob

In [4]:
chartRecommender = ChartRecommender('csv/penguins.csv', 
                                    word_embedding_dict, column_score_model, chart_type_model)

In [5]:
## rank the results by the final_score
pd.DataFrame.from_records(chartRecommender.charts).sort_values(by='final_score', ascending = False)

,chart_type,chart_type_prob,column_selection_score,final_score,indices
247,line,9.999998e-01,1.000000,9.999998e-01,"(1, 3, 6)"
232,line,1.000000e+00,0.996296,9.962965e-01,"(1, 2, 6)"
262,line,9.999995e-01,0.995994,9.959935e-01,"(1, 5, 6)"
257,line,9.999955e-01,0.995900,9.958957e-01,"(1, 4, 6)"
197,line,9.999998e-01,0.993849,9.938489e-01,"(0, 3, 6)"
182,line,1.000000e+00,0.990678,9.906777e-01,"(0, 2, 6)"
212,line,9.999994e-01,0.990501,9.905004e-01,"(0, 5, 6)"
207,line,9.999954e-01,0.989940,9.899356e-01,"(0, 4, 6)"
161,bar,9.747985e-01,0.953993,9.299513e-01,"(0, 1, 6)"
277,line,9.999994e-01,0.703667,7.036663e-01,"(2, 3, 6)"


## MV Recommender
Return a MV.
- a MV is describled as a list of charts (corresponding to each record in the above charts_df)
- current_mv: optional. 
- max_charts: number of charts in the returned MV

In [6]:
## load model
mv_model = ScoreNetLSTM(input_size=9, seq_length = 12).to(gpu)
mv_model.load_state_dict(torch.load('trainedModel/mvModel.pt', map_location=gpu))
mv_model.eval()

ScoreNetLSTM(
  (lstm): LSTM(9, 200, batch_first=True)
  (linear): Linear(in_features=2400, out_features=1, bias=True)
)

In [7]:
chartRecommender = ChartRecommender('csv/penguins.csv', 
                                    word_embedding_dict, column_score_model, chart_type_model)

In [8]:
## Recommending an MV conditioned on current_mv
current_mv = [{'indices': (1,), 'chart_type': 'pie'}]
chartRecommender.recommend_mv(mv_model, current_mv = current_mv, max_charts = len(current_mv) + 1)

[{'indices': (1,), 'chart_type': 'pie'},
 {'indices': (0, 1, 6),
  'column_selection_score': 0.953993421295783,
  'chart_type': 'bar',
  'chart_type_prob': 0.9747984895572608,
  'final_score': 0.9299513461266928}]

In [9]:
## Recommending an MV without conditions
chartRecommender.recommend_mv(mv_model, current_mv = [], max_charts = 4)

[{'indices': (2, 3, 5),
  'column_selection_score': 0.17087826005069967,
  'chart_type': 'line',
  'chart_type_prob': 0.9999961295181747,
  'final_score': 0.1708775986694998},
 {'indices': (0, 1, 6),
  'column_selection_score': 0.953993421295783,
  'chart_type': 'bar',
  'chart_type_prob': 0.9747984895572608,
  'final_score': 0.9299513461266928},
 {'indices': (2, 4, 5),
  'column_selection_score': 0.17105399002916724,
  'chart_type': 'line',
  'chart_type_prob': 0.9999778765252179,
  'final_score': 0.17105020572053245},
 {'indices': (3, 4, 5),
  'column_selection_score': 0.17125479391300252,
  'chart_type': 'line',
  'chart_type_prob': 0.9998294005573524,
  'final_score': 0.17122557794061022}]